Sign Language Detection Model developed using CNN <br>
<br>
This project makes use of the MNIST dataset, where the dataset will contain images of the sign language alphabets and numbers. Each image has size 28x28 pixels => total 784 pixels per image. <br>
<br>
![](sign_language.png)

#### Data Pre-Processing

Loading the MNIST dataset <br>
X_train and X_test contains the values of each pixel. <br>
Y_train and Y_test contains the label of each image. <br>

The image size is 28x28 hence we have to divide the array into 28x28 pixel groups. This is done by reshaping the numpy nd-arrays. 

In [3]:
import pandas as pd 
import numpy as np 

train = pd.read_csv('dataset/sign_mnist_train.csv')
test = pd.read_csv('dataset/sign_mnist_test.csv')

Y_train = train['label'].values
Y_test = test['label'].values

X_train = train.drop(['label'],axis = 1)
X_test = test.drop(['label'],axis = 1)

X_train = np.array(X_train.iloc[:,:])
X_train = np.array([np.reshape(i, (28,28)) for i in X_train])

X_test = np.array(X_test.iloc[:,:])
X_test = np.array([np.reshape(i, (28,28)) for i in X_test])

num_classes = 26
Y_train = np.array(Y_train).reshape(-1)
Y_test = np.array(Y_test).reshape(-1)

Y_train = np.eye(num_classes)[Y_train]
Y_test = np.eye(num_classes)[Y_test]

X_train = X_train.reshape((27455, 28, 28, 1))
X_test = X_test.reshape((7172, 28, 28, 1))

#### CNN Model
<br>
Resource: https://towardsdatascience.com/coding-a-convolutional-neural-network-cnn-using-keras-sequential-api-ec5211126875 <br>
<br>
Building a CNN using Keras Sequential API and using Conv2D(), MaxPooling2D()and Dense() classes to build convolutional, pooling and dense layers.<br>

A CNN can be instantiated as a Sequential model (rather than a Functional model) because each layer has exactly one input and output tensor and is stacked together to form the entire network.<br>

<b>Training Model for 50 Epochs</b>

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout


classifier = Sequential()
classifier.add(Conv2D(filters=8, kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(28,28,1),activation='relu', data_format='channels_last'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(filters=16, kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(MaxPooling2D(pool_size=(4,4)))
classifier.add(Dense(128, activation='relu'))
classifier.add(Flatten())
classifier.add(Dense(26, activation='softmax'))
classifier.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])


2022-12-05 13:37:39.101348: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.fit(X_train, Y_train, epochs=50, batch_size=100)

Epoch 1/50
275/275 [==============================] - 23s 78ms/step - loss: 2.5004 - accuracy: 0.4582
Epoch 2/50
275/275 [==============================] - 26s 96ms/step - loss: 0.6240 - accuracy: 0.7870
Epoch 3/50
275/275 [==============================] - 20s 71ms/step - loss: 0.4025 - accuracy: 0.8593
Epoch 4/50
275/275 [==============================] - 17s 63ms/step - loss: 0.2921 - accuracy: 0.8958
Epoch 5/50
275/275 [==============================] - 17s 62ms/step - loss: 0.2357 - accuracy: 0.9192
Epoch 6/50
275/275 [==============================] - 20s 73ms/step - loss: 0.1921 - accuracy: 0.9326
Epoch 7/50
275/275 [==============================] - 21s 75ms/step - loss: 0.1722 - accuracy: 0.9401
Epoch 8/50
275/275 [==============================] - 20s 74ms/step - loss: 0.1612 - accuracy: 0.9438
Epoch 9/50
275/275 [==============================] - 18s 66ms/step - loss: 0.1458 - accuracy: 0.9493
Epoch 10/50
275/275 [==============================] - 16s 56ms/step - loss: 0.125

#### Model Accuracy 

In [6]:
accuracy = classifier.evaluate(x = X_test, y = Y_test, batch_size=32)
print("Accuracy: ", accuracy[1])

225/225 [==============================] - 2s 7ms/step - loss: 0.2179 - accuracy: 0.9467
Accuracy:  0.9467372894287109


In [7]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 8)         80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        1168      
                                                                 
 dropout (Dropout)           (None, 14, 14, 16)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 16)         0         
 2D)                                                             
                                                                 
 dense (Dense)               (None, 3, 3, 128)         2

#### Saving Model

In [9]:
classifier.save('model.h5')